In [11]:
from google.colab import drive
drive.mount('/content/gdrive/')
%cd gdrive/My\ Drive/KKB-kaggle/bengaliai-cv19/notebooks

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
[Errno 2] No such file or directory: 'gdrive/My Drive/KKB-kaggle/bengaliai-cv19/notebooks'
/content/gdrive/My Drive/KKB-kaggle/bengaliai-cv19/notebooks


In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import recall_score
import cv2
# from tqdm.auto import tqdm
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import datasets, transforms, utils
from torch.autograd import Variable
import torch.optim as optim
import torchvision
import PIL
# from torchsummary import summary
import gc

dataset_dir = '../dataset'
# dataset_dir = '/kaggle/input/bengaliai-cv19'
train_df = pd.read_csv(dataset_dir + '/train.csv')
test_df = pd.read_csv(dataset_dir + '/test.csv')
class_map_df = pd.read_csv(dataset_dir + '/class_map.csv')
sample_sub_df = pd.read_csv(dataset_dir + '/sample_submission.csv')

model_dir = '../model'
# model_dir = '/kaggle/input/bengalimodels'

# 前処理を関数にまとめた
def resize(X, out_height=64, out_width=64):
    print('Resizing raw image... / 前処理実行中…')
    # resized = {} # 前処理された画像が格納されるリスト
    resized = np.zeros((len(X), out_height * out_width))

    for i in range(len(X)):
        image = X.iloc[[i]].values.reshape(HEIGHT, WIDTH)
        _, thresh = cv2.threshold(image, 30, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        contours, _ = cv2.findContours(thresh, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[-2:]
        left = 1000
        right = -1
        top = 1000
        bottom = -1

        for cnt in contours:
            x,y,w,h = cv2.boundingRect(cnt)
            left = min(x, left)
            right = max(x+w, right)
            top = min(y, top)
            bottom = max(y+h, bottom)

        roi = image[top:bottom, left:right]
        resized_roi = cv2.resize(roi, (out_height, out_width),interpolation=cv2.INTER_AREA)
        resized[i] = resized_roi.reshape(-1)

    # print(len(resized))

    return resized

def try_gpu(e):
    if torch.cuda.is_available():
        return e.cuda()
    return e

In [0]:
#resnet
def conv3x3(in_channels, out_channels, stride=1, groups=1, dilation=1):
     return nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride,
                      padding=dilation, groups=groups, bias=True,
                      dilation=dilation)

def conv1x1(in_channels, out_channels, stride=1):
     return nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=True)

class BasicBlock(nn.Module):
    #Implementation of Basic Building Block

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()

        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity_x = x  # hold input for shortcut connection

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity_x = self.downsample(x)

        out += identity_x  # shortcut connection
        return self.relu(out)

class ResidualLayer(nn.Module):

    def __init__(self, num_blocks, in_channels, out_channels, block=BasicBlock):
        super(ResidualLayer, self).__init__()
        downsample = None
        if in_channels != out_channels:
            downsample = nn.Sequential(
                conv1x1(in_channels, out_channels),
                nn.BatchNorm2d(out_channels))
        self.first_block = block(in_channels, out_channels, downsample=downsample)
        self.blocks = nn.ModuleList(block(out_channels, out_channels) for _ in range(num_blocks))

    def forward(self, x):
        out = self.first_block(x)
        for block in self.blocks:
            out = block(out)
        return out

class model(nn.Module):
    def __init__(self):
        #resnet18の実装
        super(model, self).__init__()
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = ResidualLayer(2, in_channels=64, out_channels=64)
        self.layer2 = ResidualLayer(2, in_channels=64, out_channels=128)
        self.layer3 = ResidualLayer(2, in_channels=128, out_channels=256)
        self.layer4 = ResidualLayer(2, in_channels=256, out_channels=512)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, 512)

        self.head_root = nn.Linear(512, 168) # + softmax
        self.head_vowel = nn.Linear(512, 11) # + softmax
        self.head_consonant = nn.Linear(512, 7) # + softmax
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avg_pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        head_root = self.head_root(x)
        head_vowel = self.head_vowel(x)
        head_consonant = self.head_consonant(x)

        return head_root, head_vowel, head_consonant # not sure..

model = model()
model = try_gpu(model)

criterion1 = nn.CrossEntropyLoss() 
#criterion2 = nn.CrossEntropyLoss() 
#criterion3 = nn.CrossEntropyLoss() 
optimizer = optim.Adam(model.parameters())

In [0]:
def train(model, epoch, train_loader):
    model.train()
    print(f'Epoch number {epoch}')
    for i, data in enumerate(train_loader, 0):
        inputs, root_l, vowel_l, consonant_l = data
        inputs, root_l, vowel_l, consonant_l = Variable(inputs), Variable(root_l), Variable(vowel_l), Variable(consonant_l)
        inputs, root_l, vowel_l, consonant_l = try_gpu(inputs), try_gpu(root_l), try_gpu(vowel_l), try_gpu(consonant_l)
        optimizer.zero_grad()
        root_o, vowel_o, consonant_o = model(inputs)
        loss1 = criterion1(root_o, root_l)
        loss2 = criterion1(vowel_o, vowel_l)
        loss3 = criterion1(consonant_o, consonant_l)
        (loss1+loss2+loss3).backward()
        optimizer.step()
        # if i % 500 == 0:
        #     print("epoch{} root {}/{}  loss: {}".format(epoch, i*len(inputs), len(train_loader)*len(inputs), loss1.data))
        #     print("epoch{} vowel {}/{}  loss: {}".format(epoch, i*len(inputs), len(train_loader)*len(inputs), loss2.data))
        #     print("epoch{} consonant {}/{}  loss: {}".format(epoch, i*len(inputs), len(train_loader)*len(inputs), loss3.data))

def test(model, test_loader):
    model.eval()

    size = len(test_loader.dataset)
    pred_r, pred_v, pred_c = np.zeros(size), np.zeros(size), np.zeros(size)
    true_r, true_v, true_c = np.zeros(size), np.zeros(size), np.zeros(size)
    index = 0
    
    for data in test_loader:
        inputs, root_l, vowel_l, consonant_l = data
        inputs, root_l, vowel_l, consonant_l = Variable(inputs), Variable(root_l), Variable(vowel_l), Variable(consonant_l)
        inputs, root_l, vowel_l, consonant_l = try_gpu(inputs), try_gpu(root_l), try_gpu(vowel_l), try_gpu(consonant_l)
        
        root_o, vowel_o, consonant_o = model(inputs) 
        root_pred, vowel_pred, consonant_pred = torch.max(root_o.data,1)[1], torch.max(vowel_o.data,1)[1], torch.max(consonant_o.data,1)[1]
        for i in range(inputs.size(0)):
            pred_r[index] = root_pred[i]
            pred_v[index] = vowel_pred[i]
            pred_c[index] = consonant_pred[i]
            true_r[index] = root_l[i]
            true_v[index] = vowel_l[i]
            true_c[index] = consonant_l[i]
            index += 1

    recall_r = recall_score(true_r, pred_r, average='macro')
    recall_v = recall_score(true_v, pred_v, average='macro')
    recall_c = recall_score(true_c, pred_c, average='macro')
    final_score = (2.*recall_r + recall_v + recall_c) / 4.

    print(f'Root Recall: {recall_r:.5f}')
    print(f'Vowel Recall: {recall_v:.5f}')
    print(f'Consonant Recall: {recall_c:.5f}')
    print(f'Score: {final_score:.5f}')

In [0]:
# Augmentation のための transform を定義

transform_crop = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.RandomResizedCrop((64,64), scale=(0.80, 0.90)),
                                transforms.ToTensor()
])


rotate_left = transforms.RandomAffine((-20, -10), fillcolor=255, resample=PIL.Image.BILINEAR)
rotate_right = transforms.RandomAffine((10, 20), fillcolor=255, resample=PIL.Image.BILINEAR)
transform_rotate = transforms.Compose([
                                       transforms.ToPILImage(),
                                       transforms.RandomChoice([rotate_left, rotate_right]),
                                       transforms.ToTensor()
])


class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        with_noise = tensor + torch.randn(tensor.size()) * self.std + self.mean
        return torch.max(torch.min(with_noise, torch.tensor(1.)), torch.tensor(0.))
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

transform_noise = AddGaussianNoise(0., 0.01)


# PyTorch式のデータセットクラスを定義

class MyDataset(torch.utils.data.Dataset):

    def __init__(self, X, Y, transform=None):
        self.transform = transform
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        out_data = self.X[idx].reshape(1,64,64)
        out_data = torch.tensor(out_data, dtype=torch.float)

        #root_label = torch.tensor(self.Y[0][idx], dtype=torch.long)
        #vowel_label = torch.tensor(self.Y[1][idx], dtype=torch.long)
        #cons_label = torch.tensor(self.Y[2][idx], dtype=torch.long)

        root_label = torch.tensor(np.argmax(self.Y[0][idx]), dtype=torch.long)
        vowel_label = torch.tensor(np.argmax(self.Y[1][idx]), dtype=torch.long)
        cons_label = torch.tensor(np.argmax(self.Y[2][idx]), dtype=torch.long)

        if self.transform:
            out_data = self.transform(out_data)

        return out_data, root_label, vowel_label, cons_label


class MakeSubset(torch.utils.data.Dataset):

    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        out_data, root_label, vowel_label, cons_label = self.dataset[idx]

        if self.transform:
            out_data = self.transform(out_data)

        return out_data, root_label, vowel_label, cons_label

In [17]:
model.load_state_dict(torch.load(model_dir + '/resnet_5epochs_val_saved_weights.pth'))

<All keys matched successfully>

In [18]:
# 訓練ループ / Training Loop

###############
# True なら Cross Validation を実施する
# Kaggle に提出するときは False にしてください
do_validation = True

# True なら submission.csv を生成する
create_submission = False

val_perc = 1  # validation set の割合（クロスバリデーション）
epochs = 1

###############

for parq_i in range(4):
    print('=============================')
    print(f'Parquet {parq_i} の訓練を開始')

    train_df_with_img = pd.merge(pd.read_parquet(dataset_dir + f'/train_image_data_{parq_i}.parquet'), train_df, on='image_id').drop(['image_id'], axis=1)

    HEIGHT = 137
    WIDTH = 236

    X_train = train_df_with_img.drop(columns=['grapheme_root', 'vowel_diacritic', 'consonant_diacritic', 'grapheme'])

    # X_train_resized = pd.DataFrame(resize(X_train)).T / 255.0   # 値を0~1におさめる
    # X_train_resized = X_train_resized.values
    X_train_resized = resize(X_train) / 255.0

    # メモリ節約
    del X_train

    # PyTorchのデータセットクラスを作る前に、ラベルの情報も整備
    # 1-of-K符号化とか、One Hot Encodingとか呼ばれる方法でラベルをつくる

    # 注：　PyTorchでは、ラベルはOne Hotじゃなくて良いことが判明したので、結局 MyDataset で元のラベルに戻している

    Y_train_root = pd.get_dummies(train_df_with_img['grapheme_root']).values
    Y_train_vowel = pd.get_dummies(train_df_with_img['vowel_diacritic']).values
    Y_train_cons = pd.get_dummies(train_df_with_img['consonant_diacritic']).values

    Y_train = [Y_train_root, Y_train_vowel, Y_train_cons]

    trainval_dataset = MyDataset(X_train_resized, Y_train)

    del X_train_resized
    del train_df_with_img
    gc.collect()

    if do_validation:
        n_samples = len(trainval_dataset)
        train_size = int(len(trainval_dataset)*(1.0 - val_perc))
        val_size = n_samples - train_size
        print(f'train size: {train_size}, validation size: {val_size}')

        train_dataset, val_dataset = torch.utils.data.random_split(trainval_dataset, [train_size, val_size])

        aug_dataset = MakeSubset(train_dataset)
        # train_subset2 = MakeSubset(train_dataset, transform=transform_crop)
        # train_subset3 = MakeSubset(train_dataset, transform=transform_rotate)
        # train_subset4 = MakeSubset(train_dataset, transform=transform_noise)

        del trainval_dataset
        del train_dataset
        gc.collect()

        # aug_dataset = ConcatDataset([train_subset1, train_subset2, train_subset3, train_subset4])
        
        # del train_subset1, train_subset2, train_subset3, train_subset4
        # gc.collect()

        # train_loader = DataLoader(dataset=aug_dataset, batch_size=32, shuffle=True, num_workers=4)
        val_loader = DataLoader(dataset=val_dataset, batch_size=32, num_workers=4)
        print(f'train size (after augmentation): {len(aug_dataset)}, validation size: {len(val_dataset)}')

        for i in range(1,epochs+1):
            # train(model, i, train_loader)
            test(model, val_loader)

        # メモリ節約
        del aug_dataset
        del val_dataset
        # del train_loader
        del val_loader
        gc.collect()

    else:
        train_subset1 = MakeSubset(trainval_dataset)
        train_subset2 = MakeSubset(trainval_dataset, transform=transform_crop)
        train_subset3 = MakeSubset(trainval_dataset, transform=transform_rotate)
        train_subset4 = MakeSubset(trainval_dataset, transform=transform_noise)

        del trainval_dataset
        gc.collect()

        aug_dataset = ConcatDataset([train_subset1, train_subset2, train_subset3, train_subset4])
        
        del train_subset1, train_subset2, train_subset3, train_subset4
        gc.collect()

        train_loader = DataLoader(dataset=aug_dataset, batch_size=32, shuffle=True, num_workers=4)
        print(f'train size (after augmentation): {len(aug_dataset)}')

        for i in range(1,epochs+1):
            train(model, i, train_loader)

        # メモリ節約
        del aug_dataset
        del train_loader
        gc.collect()

Parquet 0 の訓練を開始
Resizing raw image... / 前処理実行中…
train size: 0, validation size: 50210
train size (after augmentation): 0, validation size: 50210
Root Recall: 0.92546
Vowel Recall: 0.97682
Consonant Recall: 0.96116
Score: 0.94722
Parquet 1 の訓練を開始
Resizing raw image... / 前処理実行中…
train size: 0, validation size: 50210
train size (after augmentation): 0, validation size: 50210
Root Recall: 0.93119
Vowel Recall: 0.98028
Consonant Recall: 0.96265
Score: 0.95133
Parquet 2 の訓練を開始
Resizing raw image... / 前処理実行中…
train size: 0, validation size: 50210
train size (after augmentation): 0, validation size: 50210
Root Recall: 0.94201
Vowel Recall: 0.98166
Consonant Recall: 0.96731
Score: 0.95825
Parquet 3 の訓練を開始
Resizing raw image... / 前処理実行中…
train size: 0, validation size: 50210
train size (after augmentation): 0, validation size: 50210
Root Recall: 0.97964
Vowel Recall: 0.99395
Consonant Recall: 0.98890
Score: 0.98553


In [19]:
model.load_state_dict(torch.load(model_dir + '/resnet_5epochs_saved_weights.pth'))

<All keys matched successfully>

In [20]:
# 訓練ループ / Training Loop

###############
# True なら Cross Validation を実施する
# Kaggle に提出するときは False にしてください
do_validation = True

# True なら submission.csv を生成する
create_submission = False

val_perc = 1  # validation set の割合（クロスバリデーション）
epochs = 1

###############

for parq_i in range(4):
    print('=============================')
    print(f'Parquet {parq_i} の訓練を開始')

    train_df_with_img = pd.merge(pd.read_parquet(dataset_dir + f'/train_image_data_{parq_i}.parquet'), train_df, on='image_id').drop(['image_id'], axis=1)

    HEIGHT = 137
    WIDTH = 236

    X_train = train_df_with_img.drop(columns=['grapheme_root', 'vowel_diacritic', 'consonant_diacritic', 'grapheme'])

    # X_train_resized = pd.DataFrame(resize(X_train)).T / 255.0   # 値を0~1におさめる
    # X_train_resized = X_train_resized.values
    X_train_resized = resize(X_train) / 255.0

    # メモリ節約
    del X_train

    # PyTorchのデータセットクラスを作る前に、ラベルの情報も整備
    # 1-of-K符号化とか、One Hot Encodingとか呼ばれる方法でラベルをつくる

    # 注：　PyTorchでは、ラベルはOne Hotじゃなくて良いことが判明したので、結局 MyDataset で元のラベルに戻している

    Y_train_root = pd.get_dummies(train_df_with_img['grapheme_root']).values
    Y_train_vowel = pd.get_dummies(train_df_with_img['vowel_diacritic']).values
    Y_train_cons = pd.get_dummies(train_df_with_img['consonant_diacritic']).values

    Y_train = [Y_train_root, Y_train_vowel, Y_train_cons]

    trainval_dataset = MyDataset(X_train_resized, Y_train)

    del X_train_resized
    del train_df_with_img
    gc.collect()

    if do_validation:
        n_samples = len(trainval_dataset)
        train_size = int(len(trainval_dataset)*(1.0 - val_perc))
        val_size = n_samples - train_size
        print(f'train size: {train_size}, validation size: {val_size}')

        train_dataset, val_dataset = torch.utils.data.random_split(trainval_dataset, [train_size, val_size])

        aug_dataset = MakeSubset(train_dataset)
        # train_subset2 = MakeSubset(train_dataset, transform=transform_crop)
        # train_subset3 = MakeSubset(train_dataset, transform=transform_rotate)
        # train_subset4 = MakeSubset(train_dataset, transform=transform_noise)

        del trainval_dataset
        del train_dataset
        gc.collect()

        # aug_dataset = ConcatDataset([train_subset1, train_subset2, train_subset3, train_subset4])
        
        # del train_subset1, train_subset2, train_subset3, train_subset4
        # gc.collect()

        # train_loader = DataLoader(dataset=aug_dataset, batch_size=32, shuffle=True, num_workers=4)
        val_loader = DataLoader(dataset=val_dataset, batch_size=32, num_workers=4)
        print(f'train size (after augmentation): {len(aug_dataset)}, validation size: {len(val_dataset)}')

        for i in range(1,epochs+1):
            # train(model, i, train_loader)
            test(model, val_loader)

        # メモリ節約
        del aug_dataset
        del val_dataset
        # del train_loader
        del val_loader
        gc.collect()

    else:
        train_subset1 = MakeSubset(trainval_dataset)
        train_subset2 = MakeSubset(trainval_dataset, transform=transform_crop)
        train_subset3 = MakeSubset(trainval_dataset, transform=transform_rotate)
        train_subset4 = MakeSubset(trainval_dataset, transform=transform_noise)

        del trainval_dataset
        gc.collect()

        aug_dataset = ConcatDataset([train_subset1, train_subset2, train_subset3, train_subset4])
        
        del train_subset1, train_subset2, train_subset3, train_subset4
        gc.collect()

        train_loader = DataLoader(dataset=aug_dataset, batch_size=32, shuffle=True, num_workers=4)
        print(f'train size (after augmentation): {len(aug_dataset)}')

        for i in range(1,epochs+1):
            train(model, i, train_loader)

        # メモリ節約
        del aug_dataset
        del train_loader
        gc.collect()

Parquet 0 の訓練を開始
Resizing raw image... / 前処理実行中…
train size: 0, validation size: 50210
train size (after augmentation): 0, validation size: 50210
Root Recall: 0.93776
Vowel Recall: 0.97827
Consonant Recall: 0.97305
Score: 0.95671
Parquet 1 の訓練を開始
Resizing raw image... / 前処理実行中…
train size: 0, validation size: 50210
train size (after augmentation): 0, validation size: 50210
Root Recall: 0.94288
Vowel Recall: 0.98289
Consonant Recall: 0.97317
Score: 0.96045
Parquet 2 の訓練を開始
Resizing raw image... / 前処理実行中…
train size: 0, validation size: 50210
train size (after augmentation): 0, validation size: 50210
Root Recall: 0.95444
Vowel Recall: 0.98599
Consonant Recall: 0.98331
Score: 0.96955
Parquet 3 の訓練を開始
Resizing raw image... / 前処理実行中…
train size: 0, validation size: 50210
train size (after augmentation): 0, validation size: 50210
Root Recall: 0.99629
Vowel Recall: 0.99778
Consonant Recall: 0.99865
Score: 0.99725
